# Checkpoint 6: Gymnasium API and Custom Wrappers

In this notebook, we dive deep into the Gymnasium API and learn how to create custom wrappers to modify environment behavior. Wrappers are a powerful tool for preprocessing observations, shaping rewards, and adding functionality without modifying the original environment.

## Learning Objectives
- Master the Gymnasium environment API (reset, step, render, close)
- Understand observation and action spaces
- Learn the wrapper design pattern
- Implement custom wrappers for various purposes

## 1. Installation

In [ ]:
# Install required packages
!pip install -q "gymnasium[classic-control]" numpy matplotlib

## 2. Imports

In [ ]:
import gymnasium as gym
from gymnasium import spaces, Wrapper, ObservationWrapper, ActionWrapper, RewardWrapper
from gymnasium.wrappers import TimeLimit, RecordEpisodeStatistics
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from typing import Any, SupportsFloat

print("Imports successful!")
print(f"Gymnasium version: {gym.__version__}")

## 3. Theory: Gymnasium API Overview

Gymnasium provides a standardized interface for reinforcement learning environments. Every environment implements these core methods:

### Core Methods

#### `reset(seed=None, options=None) -> (observation, info)`
- Resets the environment to an initial state
- `seed`: Optional random seed for reproducibility
- `options`: Optional dict for environment-specific reset parameters
- Returns: Initial observation and info dict

#### `step(action) -> (observation, reward, terminated, truncated, info)`
- Executes one timestep in the environment
- `action`: The action to take (must be in action_space)
- Returns: 5-tuple with new state, reward, and episode status

#### `render() -> RenderFrame | list[RenderFrame] | None`
- Renders the environment for visualization
- Mode determined by `render_mode` in `gym.make()`
- Common modes: "human", "rgb_array", "ansi"

#### `close()`
- Cleans up resources (display windows, files, etc.)
- Should always be called when done with an environment

### Environment Properties

```python
env.observation_space  # Space of valid observations
env.action_space       # Space of valid actions
env.spec               # Environment specification
env.metadata           # Environment metadata (render modes, etc.)
```

## 4. Theory: Step Return Values

The `step()` method returns a 5-tuple. Understanding each component is crucial:

### `(observation, reward, terminated, truncated, info)`

| Return Value | Type | Description |
|-------------|------|-------------|
| `observation` | Depends on env | The new state after taking the action |
| `reward` | `float` | Immediate reward from the transition |
| `terminated` | `bool` | True if episode ended due to environment rules (goal reached, failure state) |
| `truncated` | `bool` | True if episode ended due to time limit or other external condition |
| `info` | `dict` | Additional diagnostic information |

### Terminated vs Truncated

**Terminated (Natural ending)**:
- CartPole: Pole fell beyond 12 degrees
- MountainCar: Car reached the goal
- Game over states

**Truncated (Artificial ending)**:
- Maximum timesteps reached (TimeLimit wrapper)
- Training budget exhausted
- External interruption

### Why the distinction matters

```python
# For bootstrapping in value-based methods:
if terminated:
    # True terminal state - no future rewards
    target = reward
elif truncated:
    # Episode cut short - should still bootstrap
    target = reward + gamma * V(next_state)
```

## 5. Theory: Spaces

Gymnasium uses `Space` objects to define valid observations and actions. Understanding spaces is essential for designing agents and wrappers.

### Common Space Types

#### `Box` - Continuous values in a bounded range
```python
# Single continuous value in [0, 1]
Box(low=0, high=1, shape=(1,), dtype=np.float32)

# 84x84 grayscale image with pixels in [0, 255]
Box(low=0, high=255, shape=(84, 84), dtype=np.uint8)

# 4D observation with different bounds per dimension
Box(low=np.array([-1, 0, -5, -np.inf]), 
    high=np.array([1, 10, 5, np.inf]), 
    dtype=np.float32)
```

#### `Discrete` - Single integer from {0, 1, ..., n-1}
```python
# 4 possible actions: 0, 1, 2, 3
Discrete(4)
```

#### `MultiDiscrete` - Multiple discrete values
```python
# Two discrete variables: first in {0,1,2}, second in {0,1,2,3,4}
MultiDiscrete([3, 5])
```

#### `Dict` - Dictionary of spaces
```python
Dict({
    'position': Box(low=-10, high=10, shape=(2,)),
    'velocity': Box(low=-1, high=1, shape=(2,)),
    'inventory': Discrete(10)
})
```

#### `Tuple` - Tuple of spaces
```python
Tuple((Discrete(3), Box(low=0, high=1, shape=(2,))))
```

### Space Methods
```python
space.sample()          # Random sample from the space
space.contains(x)       # Check if x is valid
space.shape             # Shape of samples (for Box)
space.n                 # Number of elements (for Discrete)
```

## 6. Explore Different Environment Space Types

Let's examine real environments with discrete and continuous spaces.

In [ ]:
# CartPole - Discrete action space
print("="*60)
print("CARTPOLE - DISCRETE ACTIONS")
print("="*60)

cartpole = gym.make("CartPole-v1")

print(f"\nObservation Space: {cartpole.observation_space}")
print(f"  Type: {type(cartpole.observation_space).__name__}")
print(f"  Shape: {cartpole.observation_space.shape}")
print(f"  Low bounds: {cartpole.observation_space.low}")
print(f"  High bounds: {cartpole.observation_space.high}")
print(f"  Dtype: {cartpole.observation_space.dtype}")

print(f"\nAction Space: {cartpole.action_space}")
print(f"  Type: {type(cartpole.action_space).__name__}")
print(f"  Number of actions: {cartpole.action_space.n}")
print(f"  Actions: 0=Push Left, 1=Push Right")

# Sample observation and action
print(f"\nSample observation: {cartpole.observation_space.sample()}")
print(f"Sample action: {cartpole.action_space.sample()}")

# Describe observation components
print("\nObservation components:")
print("  [0] Cart position (-4.8 to 4.8)")
print("  [1] Cart velocity (-inf to inf)")
print("  [2] Pole angle (-0.418 to 0.418 radians)")
print("  [3] Pole angular velocity (-inf to inf)")

cartpole.close()

In [ ]:
# Pendulum - Continuous action space
print("="*60)
print("PENDULUM - CONTINUOUS ACTIONS")
print("="*60)

pendulum = gym.make("Pendulum-v1")

print(f"\nObservation Space: {pendulum.observation_space}")
print(f"  Type: {type(pendulum.observation_space).__name__}")
print(f"  Shape: {pendulum.observation_space.shape}")
print(f"  Low bounds: {pendulum.observation_space.low}")
print(f"  High bounds: {pendulum.observation_space.high}")

print(f"\nAction Space: {pendulum.action_space}")
print(f"  Type: {type(pendulum.action_space).__name__}")
print(f"  Shape: {pendulum.action_space.shape}")
print(f"  Low bound: {pendulum.action_space.low}")
print(f"  High bound: {pendulum.action_space.high}")
print(f"  Action: Torque applied to pendulum (continuous from -2 to 2)")

# Sample observation and action
print(f"\nSample observation: {pendulum.observation_space.sample()}")
print(f"Sample action: {pendulum.action_space.sample()}")

print("\nObservation components:")
print("  [0] cos(theta) - x-coordinate of pendulum end")
print("  [1] sin(theta) - y-coordinate of pendulum end")
print("  [2] Angular velocity (theta_dot)")

pendulum.close()

In [ ]:
# Let's create and explore some custom spaces
print("="*60)
print("CUSTOM SPACE EXAMPLES")
print("="*60)

# MultiDiscrete space
multi_discrete = spaces.MultiDiscrete([3, 5, 2])  # 3 actions, 5 actions, 2 actions
print(f"\nMultiDiscrete([3, 5, 2]):")
print(f"  Sample: {multi_discrete.sample()}")
print(f"  nvec: {multi_discrete.nvec}")
print(f"  Valid: [0-2, 0-4, 0-1]")

# Dict space
dict_space = spaces.Dict({
    'position': spaces.Box(low=-10, high=10, shape=(2,), dtype=np.float32),
    'velocity': spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32),
    'has_key': spaces.Discrete(2)
})
print(f"\nDict space sample:")
sample = dict_space.sample()
for key, value in sample.items():
    print(f"  {key}: {value}")

# Tuple space
tuple_space = spaces.Tuple((
    spaces.Discrete(4),
    spaces.Box(low=0, high=1, shape=(3,), dtype=np.float32)
))
print(f"\nTuple space sample: {tuple_space.sample()}")

# Check if values are valid
print("\nSpace containment checks:")
print(f"  Discrete(4).contains(2): {spaces.Discrete(4).contains(2)}")
print(f"  Discrete(4).contains(5): {spaces.Discrete(4).contains(5)}")
box = spaces.Box(low=0, high=1, shape=(2,), dtype=np.float32)
print(f"  Box[0,1].contains([0.5, 0.5]): {box.contains(np.array([0.5, 0.5], dtype=np.float32))}")
print(f"  Box[0,1].contains([1.5, 0.5]): {box.contains(np.array([1.5, 0.5], dtype=np.float32))}")

## 7. Theory: The Wrapper Pattern

### What is a Wrapper?

A wrapper is a design pattern that allows you to modify an environment's behavior without changing its original code. Wrappers "wrap around" an environment and intercept/modify its methods.

### Why Use Wrappers?

1. **Modularity**: Keep environment modifications separate and reusable
2. **Composability**: Stack multiple wrappers for complex transformations
3. **Separation of concerns**: Preprocessing, logging, reward shaping in separate components
4. **Flexibility**: Easy to add/remove modifications without changing base environment

### Wrapper Hierarchy

```
gymnasium.Wrapper (base class)
    |
    |-- ObservationWrapper  (modify observations)
    |-- ActionWrapper       (modify actions)
    |-- RewardWrapper       (modify rewards)
```

### How Wrappers Work

```python
class Wrapper(gym.Env):
    def __init__(self, env):
        self.env = env  # Store the wrapped environment
        # Inherit spaces from wrapped env
        self.observation_space = env.observation_space
        self.action_space = env.action_space
    
    def step(self, action):
        return self.env.step(action)  # Delegate to wrapped env
    
    def reset(self, **kwargs):
        return self.env.reset(**kwargs)
```

### Stacking Wrappers

```python
env = gym.make("CartPole-v1")
env = WrapperA(env)  # First wrapper
env = WrapperB(env)  # Second wrapper (wraps WrapperA)
env = WrapperC(env)  # Third wrapper (wraps WrapperB)

# Call order for step():
# User -> WrapperC -> WrapperB -> WrapperA -> CartPole
# Return order:
# CartPole -> WrapperA -> WrapperB -> WrapperC -> User
```

## 8. Implement LoggingWrapper

Let's create a wrapper that tracks episode statistics like rewards, lengths, and counts.

In [ ]:
class LoggingWrapper(Wrapper):
    """
    A wrapper that logs episode statistics.
    
    Tracks:
    - Episode rewards (total per episode)
    - Episode lengths (steps per episode)
    - Episode count
    - Step count
    """
    
    def __init__(self, env: gym.Env, window_size: int = 100):
        """
        Args:
            env: The environment to wrap
            window_size: Number of recent episodes to track for moving averages
        """
        super().__init__(env)
        
        self.window_size = window_size
        
        # Statistics storage
        self.episode_rewards = deque(maxlen=window_size)
        self.episode_lengths = deque(maxlen=window_size)
        
        # Current episode tracking
        self.current_episode_reward = 0.0
        self.current_episode_length = 0
        
        # Global counters
        self.total_episodes = 0
        self.total_steps = 0
    
    def reset(self, **kwargs):
        """Reset the environment and episode tracking."""
        # Reset current episode trackers
        self.current_episode_reward = 0.0
        self.current_episode_length = 0
        
        return self.env.reset(**kwargs)
    
    def step(self, action):
        """Step the environment and update statistics."""
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Update current episode stats
        self.current_episode_reward += reward
        self.current_episode_length += 1
        self.total_steps += 1
        
        # Check if episode ended
        if terminated or truncated:
            self.episode_rewards.append(self.current_episode_reward)
            self.episode_lengths.append(self.current_episode_length)
            self.total_episodes += 1
            
            # Add episode info to info dict
            info['episode'] = {
                'reward': self.current_episode_reward,
                'length': self.current_episode_length,
                'episode_num': self.total_episodes
            }
        
        return obs, reward, terminated, truncated, info
    
    def get_statistics(self) -> dict:
        """Return current statistics."""
        stats = {
            'total_episodes': self.total_episodes,
            'total_steps': self.total_steps,
        }
        
        if len(self.episode_rewards) > 0:
            stats.update({
                'mean_reward': np.mean(self.episode_rewards),
                'std_reward': np.std(self.episode_rewards),
                'min_reward': np.min(self.episode_rewards),
                'max_reward': np.max(self.episode_rewards),
                'mean_length': np.mean(self.episode_lengths),
            })
        
        return stats
    
    def print_statistics(self):
        """Pretty print current statistics."""
        stats = self.get_statistics()
        print(f"\n{'='*40}")
        print("LOGGING WRAPPER STATISTICS")
        print(f"{'='*40}")
        print(f"Total episodes: {stats['total_episodes']}")
        print(f"Total steps: {stats['total_steps']}")
        if 'mean_reward' in stats:
            print(f"Mean reward (last {len(self.episode_rewards)}): {stats['mean_reward']:.2f} +/- {stats['std_reward']:.2f}")
            print(f"Reward range: [{stats['min_reward']:.2f}, {stats['max_reward']:.2f}]")
            print(f"Mean episode length: {stats['mean_length']:.1f}")

# Test the LoggingWrapper
print("Testing LoggingWrapper...")

env = gym.make("CartPole-v1")
env = LoggingWrapper(env)

# Run a few episodes
for episode in range(5):
    obs, info = env.reset()
    done = False
    
    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
    
    print(f"Episode {episode + 1}: Reward = {info['episode']['reward']:.1f}, Length = {info['episode']['length']}")

env.print_statistics()
env.close()

## 9. Implement ScaledRewardWrapper

A RewardWrapper subclass that scales rewards by a constant factor.

In [ ]:
class ScaledRewardWrapper(RewardWrapper):
    """
    Scales rewards by a constant factor.
    
    Useful for:
    - Normalizing rewards across different environments
    - Adjusting reward magnitude for learning stability
    - Implementing reward shaping
    """
    
    def __init__(self, env: gym.Env, scale: float = 1.0):
        """
        Args:
            env: The environment to wrap
            scale: Factor to multiply rewards by
        """
        super().__init__(env)
        self.scale = scale
    
    def reward(self, reward: SupportsFloat) -> SupportsFloat:
        """
        Transform the reward.
        
        This method is called automatically by RewardWrapper.step()
        """
        return reward * self.scale

# Test the ScaledRewardWrapper
print("Testing ScaledRewardWrapper...")
print("="*50)

# Create environments with different scales
env_normal = gym.make("CartPole-v1")
env_scaled = ScaledRewardWrapper(gym.make("CartPole-v1"), scale=0.01)

# Compare rewards
obs_normal, _ = env_normal.reset(seed=42)
obs_scaled, _ = env_scaled.reset(seed=42)

print("Taking same action in both environments:")
for step in range(5):
    action = 1  # Push right
    
    _, reward_normal, _, _, _ = env_normal.step(action)
    _, reward_scaled, _, _, _ = env_scaled.step(action)
    
    print(f"  Step {step + 1}: Normal reward = {reward_normal}, Scaled reward = {reward_scaled}")

print(f"\nScale factor: {env_scaled.scale}")
print("Notice how scaled rewards are 0.01x the normal rewards.")

env_normal.close()
env_scaled.close()

## 10. Implement NormalizeObservationWrapper

An ObservationWrapper subclass that normalizes observations to have zero mean and unit variance.

In [ ]:
class NormalizeObservationWrapper(ObservationWrapper):
    """
    Normalizes observations using running mean and standard deviation.
    
    This wrapper maintains running statistics and normalizes observations
    to have approximately zero mean and unit variance.
    """
    
    def __init__(self, env: gym.Env, epsilon: float = 1e-8, clip: float = 10.0):
        """
        Args:
            env: The environment to wrap
            epsilon: Small value to prevent division by zero
            clip: Clip normalized observations to [-clip, clip]
        """
        super().__init__(env)
        self.epsilon = epsilon
        self.clip = clip
        
        # Running statistics
        obs_shape = env.observation_space.shape
        self.running_mean = np.zeros(obs_shape, dtype=np.float64)
        self.running_var = np.ones(obs_shape, dtype=np.float64)
        self.count = 0
        
        # Update observation space to reflect normalized range
        self.observation_space = spaces.Box(
            low=-clip,
            high=clip,
            shape=obs_shape,
            dtype=np.float32
        )
    
    def observation(self, observation: np.ndarray) -> np.ndarray:
        """
        Normalize the observation.
        
        This method is called automatically by ObservationWrapper.step() and reset()
        """
        # Update running statistics (Welford's online algorithm)
        self.count += 1
        delta = observation - self.running_mean
        self.running_mean += delta / self.count
        delta2 = observation - self.running_mean
        self.running_var += (delta * delta2 - self.running_var) / self.count
        
        # Normalize
        normalized = (observation - self.running_mean) / np.sqrt(self.running_var + self.epsilon)
        
        # Clip to prevent extreme values
        return np.clip(normalized, -self.clip, self.clip).astype(np.float32)
    
    def get_statistics(self) -> dict:
        """Return current normalization statistics."""
        return {
            'mean': self.running_mean.copy(),
            'std': np.sqrt(self.running_var),
            'count': self.count
        }

# Test the NormalizeObservationWrapper
print("Testing NormalizeObservationWrapper...")
print("="*50)

env = NormalizeObservationWrapper(gym.make("CartPole-v1"))

# Collect some observations
observations = []
obs, _ = env.reset()
observations.append(obs)

for _ in range(500):
    action = env.action_space.sample()
    obs, _, terminated, truncated, _ = env.step(action)
    observations.append(obs)
    if terminated or truncated:
        obs, _ = env.reset()

observations = np.array(observations)

print(f"\nNormalized observation statistics:")
print(f"  Mean: {observations.mean(axis=0)}")
print(f"  Std:  {observations.std(axis=0)}")
print(f"  Min:  {observations.min(axis=0)}")
print(f"  Max:  {observations.max(axis=0)}")

print(f"\nObservation space bounds: [{env.observation_space.low[0]}, {env.observation_space.high[0]}]")
print("Notice observations are centered around 0 with ~unit variance.")

env.close()

## 11. Implement FrameSkipWrapper

A wrapper that repeats actions for multiple frames (common in Atari and robotics).

In [ ]:
class FrameSkipWrapper(Wrapper):
    """
    Repeats actions for a specified number of frames.
    
    Benefits:
    - Reduces decision frequency (faster training)
    - Encourages temporally extended actions
    - Accumulates rewards over skipped frames
    """
    
    def __init__(self, env: gym.Env, skip: int = 4):
        """
        Args:
            env: The environment to wrap
            skip: Number of times to repeat each action
        """
        super().__init__(env)
        self.skip = skip
    
    def step(self, action):
        """
        Repeat action for `skip` frames, accumulating reward.
        """
        total_reward = 0.0
        terminated = False
        truncated = False
        info = {}
        
        for _ in range(self.skip):
            obs, reward, terminated, truncated, info = self.env.step(action)
            total_reward += reward
            
            # Stop early if episode ends
            if terminated or truncated:
                break
        
        return obs, total_reward, terminated, truncated, info

# Test the FrameSkipWrapper
print("Testing FrameSkipWrapper...")
print("="*50)

env_normal = gym.make("CartPole-v1")
env_skip = FrameSkipWrapper(gym.make("CartPole-v1"), skip=4)

# Compare step counts
def run_episode(env, seed=42):
    obs, _ = env.reset(seed=seed)
    total_reward = 0
    steps = 0
    done = False
    
    while not done:
        action = 1  # Always push right for consistency
        obs, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        steps += 1
        done = terminated or truncated
    
    return total_reward, steps

reward_normal, steps_normal = run_episode(env_normal)
reward_skip, steps_skip = run_episode(env_skip)

print(f"Normal environment:")
print(f"  Steps taken: {steps_normal}")
print(f"  Total reward: {reward_normal}")

print(f"\nFrame-skip (skip=4) environment:")
print(f"  Steps taken: {steps_skip}")
print(f"  Total reward: {reward_skip}")
print(f"  Effective frames: ~{steps_skip * 4}")

print(f"\nStep reduction: {steps_normal / steps_skip:.1f}x fewer decisions")
print("Note: Rewards are similar because frame-skip accumulates rewards.")

env_normal.close()
env_skip.close()

## 12. Demonstrate Wrapper Composition

Let's stack multiple wrappers together to see how they compose.

In [ ]:
print("="*60)
print("WRAPPER COMPOSITION DEMONSTRATION")
print("="*60)

# Create a heavily wrapped environment
# Order matters! Wrappers are applied from inside out

# Start with base environment
env = gym.make("CartPole-v1")
print(f"\n1. Base environment: {type(env).__name__}")

# Add frame skip (reduces step frequency)
env = FrameSkipWrapper(env, skip=2)
print(f"2. Add FrameSkipWrapper (skip=2): {type(env).__name__}")

# Add observation normalization
env = NormalizeObservationWrapper(env)
print(f"3. Add NormalizeObservationWrapper: {type(env).__name__}")

# Add reward scaling
env = ScaledRewardWrapper(env, scale=0.1)
print(f"4. Add ScaledRewardWrapper (scale=0.1): {type(env).__name__}")

# Add logging
env = LoggingWrapper(env)
print(f"5. Add LoggingWrapper: {type(env).__name__}")

# Show the wrapper chain
print("\n" + "-"*40)
print("Wrapper chain (outer to inner):")
current = env
depth = 0
while hasattr(current, 'env'):
    print(f"  {'  ' * depth}{type(current).__name__}")
    current = current.env
    depth += 1
print(f"  {'  ' * depth}{type(current).__name__} (base)")

# Test the composed environment
print("\n" + "-"*40)
print("Testing composed environment:")

obs, _ = env.reset(seed=42)
print(f"\nInitial observation (normalized): {obs}")

# Run one episode
done = False
step_count = 0

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    step_count += 1
    done = terminated or truncated
    
    if step_count <= 3:
        print(f"\nStep {step_count}:")
        print(f"  Observation: {obs}")
        print(f"  Reward (scaled): {reward}")

print(f"\nEpisode complete!")
if 'episode' in info:
    print(f"Total reward: {info['episode']['reward']:.2f}")
    print(f"Total steps: {info['episode']['length']}")

env.close()

## 13. Test Wrapped Environment

Let's run multiple episodes with our composed environment and analyze the statistics.

In [ ]:
# Create our full wrapper stack
def create_wrapped_env(seed=None):
    env = gym.make("CartPole-v1")
    env = FrameSkipWrapper(env, skip=2)
    env = NormalizeObservationWrapper(env)
    env = ScaledRewardWrapper(env, scale=0.1)
    env = LoggingWrapper(env)
    return env

env = create_wrapped_env()

print("Running 20 episodes with wrapped environment...")
print("="*50)

episode_rewards = []
episode_lengths = []

for episode in range(20):
    obs, _ = env.reset(seed=episode)
    done = False
    episode_reward = 0
    
    while not done:
        # Simple policy: move in direction of pole lean
        # obs[2] is pole angle (negative = leaning left)
        action = 1 if obs[2] > 0 else 0
        
        obs, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        done = terminated or truncated
    
    episode_rewards.append(info['episode']['reward'])
    episode_lengths.append(info['episode']['length'])
    
    if (episode + 1) % 5 == 0:
        print(f"Episode {episode + 1}: Reward = {info['episode']['reward']:.2f}, Length = {info['episode']['length']}")

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(episode_rewards, 'b-', linewidth=2)
axes[0].axhline(y=np.mean(episode_rewards), color='r', linestyle='--', label=f'Mean: {np.mean(episode_rewards):.2f}')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Episode Reward (scaled)')
axes[0].set_title('Episode Rewards')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(episode_lengths, 'g-', linewidth=2)
axes[1].axhline(y=np.mean(episode_lengths), color='r', linestyle='--', label=f'Mean: {np.mean(episode_lengths):.1f}')
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Episode Length (wrapper steps)')
axes[1].set_title('Episode Lengths')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('wrapper_test_results.png', dpi=100, bbox_inches='tight')
plt.show()

# Print final statistics from LoggingWrapper
env.print_statistics()
env.close()

## 14. Built-in Gymnasium Wrappers

Gymnasium provides many useful built-in wrappers. Let's explore some important ones.

In [ ]:
print("="*60)
print("BUILT-IN GYMNASIUM WRAPPERS")
print("="*60)

# 1. TimeLimit - Truncates episodes after max steps
print("\n1. TimeLimit Wrapper")
print("-" * 40)

from gymnasium.wrappers import TimeLimit

# Create environment with custom time limit
env = gym.make("CartPole-v1")
env = TimeLimit(env, max_episode_steps=50)  # Override default 500

obs, _ = env.reset(seed=42)
steps = 0
done = False

while not done:
    obs, _, terminated, truncated, _ = env.step(1)  # Always push right
    steps += 1
    done = terminated or truncated

print(f"  Episode ended after {steps} steps")
print(f"  Terminated (natural): {terminated}")
print(f"  Truncated (time limit): {truncated}")
env.close()

# 2. RecordEpisodeStatistics - Tracks episode stats automatically
print("\n2. RecordEpisodeStatistics Wrapper")
print("-" * 40)

from gymnasium.wrappers import RecordEpisodeStatistics

env = gym.make("CartPole-v1")
env = RecordEpisodeStatistics(env, buffer_length=10)

# Run a few episodes
for ep in range(5):
    obs, _ = env.reset()
    done = False
    while not done:
        obs, _, terminated, truncated, info = env.step(env.action_space.sample())
        done = terminated or truncated
    
    # Episode stats are in info when episode ends
    print(f"  Episode {ep + 1}: r={info['episode']['r']:.1f}, l={info['episode']['l']}, t={info['episode']['t']:.2f}s")

env.close()

In [ ]:
# 3. More built-in wrappers demonstration
print("\n3. Other Useful Built-in Wrappers")
print("-" * 40)

from gymnasium.wrappers import (
    FlattenObservation,
    GrayscaleObservation,
    ResizeObservation,
    ClipAction,
    RescaleAction,
)

print("""
Available wrappers for different purposes:

OBSERVATION WRAPPERS:
  - FlattenObservation: Flattens dict/tuple observations
  - GrayscaleObservation: Converts RGB to grayscale (for images)
  - ResizeObservation: Resizes image observations
  - NormalizeObservation: Normalizes observations (running stats)
  - FrameStack: Stacks multiple frames (built into SB3)

ACTION WRAPPERS:
  - ClipAction: Clips continuous actions to valid range
  - RescaleAction: Rescales action space to new range

REWARD WRAPPERS:
  - ClipReward: Clips rewards to a range
  - NormalizeReward: Normalizes rewards (running stats)

UTILITY WRAPPERS:
  - TimeLimit: Truncates episodes after N steps
  - RecordEpisodeStatistics: Tracks reward/length stats
  - RecordVideo: Records episodes as video files
  - AutoResetWrapper: Automatically resets on done
""")

# Example: RescaleAction
print("\nExample: RescaleAction with Pendulum")
print("-" * 40)

env = gym.make("Pendulum-v1")
print(f"Original action space: {env.action_space}")
print(f"  Range: [{env.action_space.low[0]}, {env.action_space.high[0]}]")

# Rescale to [-1, 1]
env_rescaled = RescaleAction(env, min_action=-1.0, max_action=1.0)
print(f"\nRescaled action space: {env_rescaled.action_space}")
print(f"  Range: [{env_rescaled.action_space.low[0]}, {env_rescaled.action_space.high[0]}]")

# Action 0.5 in rescaled space becomes 1.0 in original space
print("\nAction mapping (rescaled -> original):")
print("  -1.0 -> -2.0")
print("   0.0 ->  0.0")
print("   1.0 ->  2.0")

env.close()
env_rescaled.close()

## 15. Quiz: Test Your Understanding

Answer these questions to check your understanding of the Gymnasium API and wrappers.

---

**Question 1**: What is the difference between `terminated` and `truncated` in the step return?

A) They mean the same thing - the episode is over  
B) `terminated` means natural ending (goal/failure), `truncated` means artificial ending (time limit)  
C) `truncated` is for training, `terminated` is for evaluation  
D) `terminated` is for discrete actions, `truncated` is for continuous actions

---

**Question 2**: Which space type would you use for a robot arm with 6 continuous joint angles, each between -pi and pi?

A) `Discrete(6)`  
B) `MultiDiscrete([6])`  
C) `Box(low=-pi, high=pi, shape=(6,))`  
D) `Tuple((Discrete(6), Discrete(6)))`

---

**Question 3**: When stacking wrappers, which wrapper processes the action first?

A) The innermost (closest to base environment)  
B) The outermost (furthest from base environment)  
C) All wrappers process simultaneously  
D) It depends on the wrapper type

---

**Question 4**: Which parent class should you use to create a wrapper that modifies observations?

A) `gym.Wrapper`  
B) `gym.ObservationWrapper`  
C) `gym.ActionWrapper`  
D) `gym.RewardWrapper`

---

**Question 5**: Why is frame skipping useful in reinforcement learning?

A) It makes the game graphics look better  
B) It reduces decision frequency and encourages temporally extended actions  
C) It increases the observation space size  
D) It is only used for debugging

---

### Answers

<details>
<summary>Click to reveal answers</summary>

1. **B** - `terminated` indicates the episode ended due to the environment's rules (like reaching a goal or failure state), while `truncated` indicates it ended due to external factors like a time limit. This distinction matters for value bootstrapping in RL algorithms.

2. **C** - `Box` is the correct choice for continuous values with defined bounds. `Box(low=-pi, high=pi, shape=(6,))` creates a 6-dimensional continuous space where each dimension is bounded between -pi and pi.

3. **B** - The outermost wrapper receives the action first. The call chain goes: User -> Outer wrapper -> ... -> Inner wrapper -> Base environment. Return values go in reverse order.

4. **B** - `ObservationWrapper` is designed specifically for modifying observations. You only need to implement the `observation()` method, and the wrapper handles calling it at the right times.

5. **B** - Frame skipping reduces the number of decisions the agent needs to make (4x fewer with skip=4) and encourages actions that have effects over multiple timesteps, which can speed up training and improve performance.

</details>

## 16. Cleanup

In [ ]:
# Clean up any remaining environments
import gc

# Force garbage collection
gc.collect()

print("Cleanup complete!")
print("\n" + "="*60)
print("CHECKPOINT 6 COMPLETE!")
print("="*60)
print("\nKey Takeaways:")
print("1. Gymnasium provides a standardized API: reset(), step(), render(), close()")
print("2. step() returns (obs, reward, terminated, truncated, info)")
print("3. Spaces define valid observations/actions (Box, Discrete, etc.)")
print("4. Wrappers modify environment behavior without changing source code")
print("5. Specialized wrappers exist: ObservationWrapper, RewardWrapper, ActionWrapper")
print("6. Wrappers can be composed/stacked for complex transformations")
print("\nNext: Apply these concepts to build custom N64 Mario Kart environments!")